<a href="https://colab.research.google.com/github/100894894/Visual/blob/main/Week2e_Tutorial_OneHot_F23.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Load Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [2]:
#Load Dataset
data=pd.read_csv('./InsRaw.csv')
data.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [3]:
#Show Key Statistics
data.describe()

,age,bmi,children,charges
count,1338.000000,1338.000000,1338.000000,1338.000000
mean,39.207025,30.663397,1.094918,13270.422265
std,14.049960,6.098187,1.205493,12110.011237
min,18.000000,15.960000,0.000000,1121.873900
25%,27.000000,26.296250,0.000000,4740.287150
50%,39.000000,30.400000,1.000000,9382.033000
75%,51.000000,34.693750,2.000000,16639.912515
max,64.000000,53.130000,5.000000,63770.428010


In [4]:
#Create One Hot Encoded Dataframe

from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(drop='first')

data_object = data.select_dtypes('object')
ohe.fit(data_object)

codes = ohe.transform(data_object).toarray()
feature_names = ohe.get_feature_names_out(data_object.columns)

data_ohe = pd.concat([data.select_dtypes(exclude='object'),
               pd.DataFrame(codes,columns=feature_names).astype(int)], axis=1)

data_ohe.head()

,age,bmi,children,charges,sex_male,smoker_yes,region_northwest,region_southeast,region_southwest
0,19,27.900,0,16884.92400,0,1,0,0,1
1,18,33.770,1,1725.55230,1,0,0,1,0
2,28,33.000,3,4449.46200,1,0,0,1,0
3,33,22.705,0,21984.47061,1,0,1,0,0
4,32,28.880,0,3866.85520,1,0,1,0,0


In [5]:
#Define x and y variable
x = data_ohe.drop(['charges'],axis=1).to_numpy()
y = data_ohe['charges'].to_numpy()

#Create Train and Test Datasets
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=100)

#Scale the Data
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train2 = sc.fit_transform(x_train)
x_test2 = sc.transform(x_test)

#Model
from sklearn.linear_model import LinearRegression

In [6]:
#Create Standard Model - using Dummy Variables

from sklearn import metrics

for name,method in [('Linear regression', LinearRegression())]:
    method.fit(x_train2,y_train)
    predict = method.predict(x_test2)

print('\n Regression Model - using OneHotEncoder')
print('\nMethod: {}'.format(name))

#Coefficents
print('\nIntercept: {:.2f}'.format(float(method.intercept_)))
coeff_table=pd.DataFrame(np.transpose(method.coef_),
                         data_ohe.drop(['charges'],axis=1).columns,
                         columns=['Coefficients'])
print(coeff_table)

#R2,MAE,MSE and RMSE
print('\nR2: {:.2f}'.format(metrics.r2_score(y_test,predict)))
adjusted_r_squared = 1-(1-metrics.r2_score(y_test,predict))*(len(y)-1)/(len(y)-x.shape[1]-1)
print('Adj_R2: {:0.2f}'.format(adjusted_r_squared))
print('Mean Absolute Error: {:.2f}'.format(metrics.mean_absolute_error(y_test, predict)))
print('Mean Squared Error: {:.2f}'.format(metrics.mean_squared_error(y_test, predict)))
print('Root Mean Squared Error: {:.2f}'.format(np.sqrt(metrics.mean_squared_error(y_test, predict))))


 Regression Model - using OneHotEncoder

Method: Linear regression

Intercept: 13064.90
                  Coefficients
age                3716.450753
bmi                1919.158871
children            562.801388
sex_male              6.307362
smoker_yes         9347.494050
region_northwest   -174.226169
region_southeast   -409.493061
region_southwest   -479.883150

R2: 0.79
Adj_R2: 0.79
Mean Absolute Error: 3916.44
Mean Squared Error: 32193193.04
Root Mean Squared Error: 5673.90
